In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
import cv2
import os
import numpy as np
import random
import math
from math import log, e
import matplotlib.pyplot as plt
from PIL import Image
from numpy import divide, int8, multiply, ravel, sort, zeros_like

In [41]:
#read image
noah = cv2.imread("/content/drive/MyDrive/Colab Notebooks/IP-HW3/Noah.JPG", cv2.IMREAD_GRAYSCALE) 
print(noah.shape) #確認pic size
print(noah.size) #確認pixel數量
value,counts = np.unique(noah, return_counts=True) #計算每個gray value的數量
print(counts)
print(value)
print(noah)

(1773, 2364)
4191372
[   793   1365  13724  30020  27915  23130  18856  16283  14779  13167
  11972  11270  10401  10003   9704   9616   9731   9771  10153  10272
  10327  10953  11358  11686  11890  12389  12305  12654  12397  12421
  12209  11995  11632  11454  11019  11176  11166  11061  10799  10774
  10655  10564  10666  10648  10483  10494  10513  10488  10563  10200
  10151  10135   9925   9868   9632   9324   9346   8870   8741   8422
   8116   7781   7880   7550   7742   7623   7301   7323   7371   7240
   7222   7247   7115   7134   7139   6977   7179   7269   7051   6993
   7066   7085   6931   6840   6875   6569   6762   6500   6480   6201
   6069   6200   5926   5938   5802   5827   5937   5814   5742   5643
   5430   5164   5053   4804   4881   4729   4585   4445   4306   4111
   4111   3940   4015   3845   3770   3760   3754   3508   3579   3535
   3467   3446   3423   3300   3460   3215   3499   3273   3379   3350
   3322   3437   3435   3415   3640   3650   3680   3831

In [42]:
from google.colab.patches import cv2_imshow
def gaussian_noise(img, mean, sigma):

    # int -> float (標準化)
    img = img / 255
    # 隨機生成高斯 noise (float + float)
    noise = np.random.normal(mean, sigma, img.shape)
    # noise + 原圖
    gaussian_out = img + noise
    # 所有值必須介於 0~1 之間，超過1 = 1，小於0 = 0
    gaussian_out = np.clip(gaussian_out, 0, 1)

    # 原圖: float -> int (0~1 -> 0~255)
    gaussian_out = np.uint8(gaussian_out*255)
    # noise: float -> int (0~1 -> 0~255)
    noise = np.uint8(noise*255)

    print("gaussian noise: ")
    cv2_imshow(noise)

    print("Picture add gaussian noise: ")
    cv2_imshow(gaussian_out)

    return gaussian_out

In [43]:
noiseNoah = gaussian_noise(noah, 0, 0.5)
cv2.imwrite('/content/drive/MyDrive/Colab Notebooks/IP-HW3/noiseNoah.jpg', noiseNoah)

Output hidden; open in https://colab.research.google.com to view.

In [49]:
#Padding 3x3 
paddedNoiseNoah = cv2.copyMakeBorder(noiseNoah,1,1,1,1,cv2.BORDER_CONSTANT,value= 0)
print(paddedNoiseNoah)
print(paddedNoiseNoah.shape)
value,counts = np.unique(paddedNoiseNoah, return_counts=True) 
print(counts)
print(value)
cv2_imshow(paddedNoiseNoah) 
cv2.imwrite('/content/drive/MyDrive/Colab Notebooks/IP-HW3/paddedNoiseNoah.jpg', paddedNoiseNoah)

Output hidden; open in https://colab.research.google.com to view.

In [50]:
#median filter
def median_filter(gray_img, mask):
    """
    :param gray_img: gray image
    :param mask: mask size
    :return: image with median filter
    """
    # set image borders
    bd = int(mask / 2)
    # copy image size
    median_img = zeros_like(gray_img)
    for i in range(bd, gray_img.shape[0] - bd):
        for j in range(bd, gray_img.shape[1] - bd):
            # get mask according with mask
            kernel = ravel(gray_img[i - bd : i + bd + 1, j - bd : j + bd + 1])
            # calculate mask median
            median = sort(kernel)[int8(divide((multiply(mask, mask)), 2) + 1)]
            median_img[i, j] = median
    return median_img

In [53]:
#do median filter 3x3 first time
filteredNoah3x3 = median_filter(paddedNoiseNoah,3)
print(filteredNoah3x3)
print(filteredNoah3x3.shape)
cv2_imshow(filteredNoah3x3)

Output hidden; open in https://colab.research.google.com to view.

In [54]:
#do median filter 3x3 second time
filteredNoah3x3x2 = median_filter(filteredNoah3x3,3)
print(filteredNoah3x3x2)
print(filteredNoah3x3x2.shape)
cv2_imshow(filteredNoah3x3x2)
cv2.imwrite('/content/drive/MyDrive/Colab Notebooks/IP-HW3/filteredNoah3x3twice.jpg', filteredNoah3x3x2)

Output hidden; open in https://colab.research.google.com to view.

In [65]:
#crop the excess 0 padding
Noah3x3 = [[0]*noah.shape[0] for _ in range(noah.shape[1])] #initialize shape
print(filteredNoah3x3x2)
print(filteredNoah3x3x2.shape)
#crop from output of median filter
Noah3x3 = filteredNoah3x3x2[1:filteredNoah3x3x2.shape[0]-1,1:filteredNoah3x3x2.shape[1]-1] 
print(Noah3x3)
print(Noah3x3.shape)
cv2_imshow(Noah3x3)

Output hidden; open in https://colab.research.google.com to view.

In [59]:
#Padding 5x5 
paddedNoiseNoah5x5 = cv2.copyMakeBorder(noiseNoah,2,2,2,2,cv2.BORDER_CONSTANT,value= 0)
print(paddedNoiseNoah5x5)
print(paddedNoiseNoah5x5.shape)
value,counts = np.unique(paddedNoiseNoah5x5, return_counts=True) 
print(counts)
print(value)
cv2_imshow(paddedNoiseNoah5x5) 
cv2.imwrite('/content/drive/MyDrive/Colab Notebooks/IP-HW3/paddedNoiseNoah5x5.jpg', paddedNoiseNoah5x5)

Output hidden; open in https://colab.research.google.com to view.

In [60]:
#do median filter 5x5
filteredNoah5x5 = median_filter(paddedNoiseNoah5x5,5)
print(filteredNoah5x5)
print(filteredNoah5x5.shape)
cv2_imshow(filteredNoah5x5)

Output hidden; open in https://colab.research.google.com to view.

In [66]:
#crop the excess 0 padding
Noah5x5 = [[0]*noah.shape[0] for _ in range(noah.shape[1])] #initialize shape
print(filteredNoah5x5)
print(filteredNoah5x5.shape)
#crop from output of median filter
Noah5x5 = filteredNoah5x5[2:filteredNoah5x5.shape[0]-2,2:filteredNoah5x5.shape[1]-2] 
print(Noah5x5)
print(Noah5x5.shape)
cv2_imshow(Noah5x5)

Output hidden; open in https://colab.research.google.com to view.

In [61]:
def psnr(img1, img2):
  mse = np.mean((img1/1.0 - img2/1.0) **2)
  if mse < 1.0e-10:
    return 100
  return 10 * math.log10(255.0**2/mse)

In [72]:
cv2_imshow(Noah3x3)
cv2_imshow(Noah5x5)
print(psnr(noah,Noah3x3))
print(psnr(noah,Noah5x5))

Output hidden; open in https://colab.research.google.com to view.